# Recuperación de pacientes del archivo de XiO
---------
### Tests locales
## Procedimiento
- Configurar las herramientas necesarias
- Recuperar mediante FTP los datos archivados de XiO (herramientas externas)
- Subir a **gDrive** de Radiofísica los datos archivados
- Introducir los parámetros relevantes
- Correr los scripts de recuparación
- Descargar los archivos recuperados
- Limpiar en gDrive los archivos creados
## Configuración de herramientas

In [1]:
# Importación de módulos
# - Manipulación de archivos DICOM
import pydicom as dicom
# - Conversión del formato nativo de XiO a DICOM
import pyplastimatch as plast
# - Rutas y comprobaciones de existencia de ficheros 
from os import path
from pathlib import Path
from glob import glob
from sys import path as syspath

In [2]:
from XiORecovery.scripts.dcmcoher import correctImagePositionPatientInCTImages, correctRTPlan

Crear uan referencia al directorio base de trabajo, crearlo e ir a él

In [13]:
%%capture
# Referenciar
baseworkdir = '/home/radiofisica/devel'
baseworkdir = baseworkdir + '/patient'

# Crear el directorio
!mkdir -p $baseworkdir

# Ir al directorio base
%cd $baseworkdir

Crear la carpeta `scripts` en el directorio base de trabajo en **gDrive** y descargar los scripts que complementan el funcionamiento de `plastimatch`

In [14]:
%%capture
# Descargar los scripts de corrección
if Path(baseworkdir + '/scripts').exists():
  !rm -rf $baseworkdir/scripts
!mkdir -p $baseworkdir/scripts
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/xioss
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/genindex
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/deltact
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/dcmcoher.py


# Añadir al sistema la ruta de los scripts
if baseworkdir + '/scripts' not in syspath:
  syspath.insert(0, baseworkdir + '/scripts')

# Comprobar si el plan genérico con un único campo está y si no descargarlo
if not Path('RTPlan.dcm').exists():
  !wget https://raw.githubusercontent.com/csarux/XiOrecovery/main/DICOMSets/RTPlan.dcm

In [15]:
ArchivedPatient = '100668_Bravo_Conejo_Elisa_Jul_08_2013.tgz'

In [16]:
!tar -xvzf /home/radiofisica/Shares/Radiofisica/csr/XiO/patient/$ArchivedPatient

100668_Bravo_Conejo_Elisa_Jul_08_2013.tgz
./archive_configuration
./patient/100668/
./patient/100668/.dirlock
./patient/100668/pdf/
./patient/100668/pdf/promoted_pdf
./patient/100668/.focalts
./patient/100668/dvh/
./patient/100668/dvh/plan.p.MDFASE1
./patient/100668/dvh/plan.p.SUMA
./patient/100668/dvh/plan.t.272
./patient/100668/dvh/plan.p.MIFASE1
./patient/100668/dvh/plan.p.MDFASE2
./patient/100668/dvh/plan.p.MIFASE2
./patient/100668/demographic
./patient/100668/plan/
./patient/100668/plan/MIFASE1/
./patient/100668/plan/MIFASE1/tel.1.dose.5
./patient/100668/plan/MIFASE1/dose.1
./patient/100668/plan/MIFASE1/tel.1
./patient/100668/plan/MIFASE1/tel.1.dose.2
./patient/100668/plan/MIFASE1/tel.1.dose.6
./patient/100668/plan/MIFASE1/tel.1.dose.1
./patient/100668/plan/MIFASE1/plan
./patient/100668/plan/MIFASE1/tel.1.dose.3
./patient/100668/plan/MIFASE1/tel.1.dose.4
./patient/100668/plan/MIFASE1/dirdosevalid
./patient/100668/plan/MDFASE2/
./patient/100668/plan/MDFASE2/dose.1
./patient/100668/

In [17]:
PatientID = ArchivedPatient.split('_')[0]
PatientID

'100668'

In [18]:
demographicfile = !find ./patient/$PatientID -name demographic -print 
with open(demographicfile[0]) as demogf:
  demographicdata = demogf.read()

ApellidosNombre = demographicdata.split('\n')[2]
ApellidosNombre

'Bravo Conejo,Elisa'

In [4]:
!tree -d patient

patient
└── 100668
    ├── anatomy
    │   └── studyset
    │       └── CT1
    │           ├── DCMData
    │           ├── ss
    │           └── surfaces
    ├── dvh
    ├── pdf
    └── plan
        ├── MDFASE1
        ├── MDFASE2
        ├── MIFASE1
        ├── MIFASE2
        ├── SETUPMDFASE1
        ├── SETUPMDFASE2
        ├── SETUPMIFASE1
        ├── SETUPMIFASE2
        └── SUMA

19 directories


In [19]:
CT = 'CT1'
Plan ='MDFASE1'

## Correr los scripts de recuperación
**Estrategia**

- Crear el índice de las imágenes de CT si no existe
- Recuperar las estructuras a partir de los archivos de texto `.msh` 
- Transformar estructuras y dosis a formato DICOM
- Introducir un plan básico genérico y asociarle a la dosis recpuerada y al conjunto de estructuras
- Establecer una marco de referencia coherente entre las imágenes CT y el conjunto plan, dosis, estructuras

In [20]:
# Si no existe crear el índice de imágenes CT y las máscaras `.mha` de todas las estructuras
if not Path(baseworkdir + '/patient/' + PatientID + '/anatomy/studyset/' + CT + '/index.dat').exists():
  # Copiar el script para la creación del índice de cortes de CT
  !cp scripts/genindex patient/$PatientID/anatomy/studyset/$CT

  # Cambiar el directorio de trabajo
  %cd patient/$PatientID/anatomy/studyset/$CT

  # Adjudicar permisos de ejecución y crear el índice
  !chmod +x genindex 
  !./genindex

  # Crear la carpeta para los archivos máscara con las estructuras
  !mkdir -p ./ss

  # Copiar el script para la creación de las estructuras
  !cp $baseworkdir/scripts/xioss .

  # Correr el script de recuperación de las estructuras
  !chmod +x xioss
  !./xioss --input $baseworkdir/patient/$PatientID --plan $Plan > xioss.log

  # Cambiar el directorio de trabajo al raíz de los pacientes
  %cd $baseworkdir/patient

  # Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
  !cp $baseworkdir/scripts/deltact .

  # Adjudicar permisos de ejecución y calcular el delta
  !chmod +x deltact
  !./deltact --input $PatientID --studyset $CT 
  

# Regresar al directorio base
%cd $baseworkdir

# Variables de pyplastimatch
verbose = True
path_to_logfile_xiodcm = './' + PatientID + '_xiodcm.log'

# Opciones de plastimatch convert
convert_args_rt ={'input' :  'patient/' + PatientID ,
                  'output-dicom' : 'xiodcm'} 

# Correr el script de conversión de los datos XiO en estudio DICOM                   
plast.convert(verbose=verbose, path_to_log_file=path_to_logfile_xiodcm, **convert_args_rt )

# Extracción del delta entre los marcos de referencia del estudio CT y el plan a partir del archivo CT#.infor
## Cambiar el directorio de trabajo al raíz de los pacientes
%cd $baseworkdir/patient

# Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
!cp $baseworkdir/scripts/deltact .

## Asegurar que el archivo se actualiza
if Path('./deltaframes').exists():
  !rm -f ./deltaframes
## Adjudicar permisos de ejecución y calcular el delta
!chmod +x deltact
!./deltact --input $PatientID --studyset $CT > deltact.log

## Regresar al directorio base
%cd $baseworkdir

# Incorporar el plan genérico 
!cp RTPlan.dcm xiodcm

# Importar las herramientas de corrección
# from dcmcoher import correctImagePositionPatientInCTImages, correctRTPlan

# Asociar al plan las dosis y estructuras
correctRTPlan()

# Establecer el mismo marco de referencia en todo el estudio
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT)

/home/radiofisica/Development/patient/patient/100668/anatomy/studyset/CT1
mv: 'ss/patient.mha' and 'ss/patient.mha' are the same file
mv: 'ss/medula.mha' and 'ss/medula.mha' are the same file
mv: 'ss/PYV MAMA D.mha' and 'ss/PYV MAMA D.mha' are the same file
Use of uninitialized value in string eq at ./xioss line 151.
Consistency lack in structure match between XiO contour file and plastimatch prefix output at ./xioss line 151.
/home/radiofisica/Development/patient/patient
Reading CT1.info file
Reference frame delta x[mm], y[mm]: -0.000001,-199.000000
Writing delta to deltaframes file
Finished!
/home/radiofisica/Development/patient

Running 'plastimatch convert' with the specified arguments:
  --input patient/100668
  --output-dicom xiodcm
... Done.
/home/radiofisica/Development/patient/patient
/home/radiofisica/Development/patient


Limpiar el directorio temporal de trabajo

In [22]:
!rm -rf 100668_xiodcm.log  archive_configuration  patient  RTPlan.dcm  scripts  xiodcm